In [1]:
import pandas as pd

# Cargue de data frames

nrows para subir muestras

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/megelon/datasets/master/Historical%20Product%20Demand.csv', 
                    sep=',',usecols=['Product_Code','Warehouse', 'Product_Category', 'Date', 'Order_Demand'],
                    dtype={'Product_Code':str, 'Warehouse':str, 'Product_Category':str, 'Date':str, 'Order_Demand':int},
                    nrows=100)

# Conexiones

In [3]:
import psycopg2
import getpass
conn=psycopg2.connect(
        user="postgres",
        password=getpass.getpass("Enter your secret: "),
        host="localhost",
        port=5432,
        database="postgres")
cursor=conn.cursor()

/home/megelon/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Enter your secret:  ··········


# Create Table

In [36]:
cursor.execute(
"""
CREATE TABLE prodemand 
(
Product_Code VARCHAR(12) NOT NULL,
Warehouse VARCHAR(12) NOT NULL,
Product_Category VARCHAR(12) NOT NULL,
Date VARCHAR(12) NOT NULL,
Order_Demand int NOT NULL
--,PRIMARY KEY (Product_Code)
)
;
"""
)

# INSERT

In [37]:
insert_into="""
INSERT INTO prodemand 
(
Product_Code,Warehouse,Product_Category,Date,Order_Demand
)
VALUES(
'{Product_Code}',
'{Warehouse}',
'{Product_Category}',
'{Date}',
'{Order_Demand}'
);
"""

# CURSOR

In [38]:
from tqdm import tqdm

In [39]:
#insertar variables
for idx, row in tqdm(df.iterrows()):
    cursor.execute(insert_into.format(
        Product_Code= row['Product_Code'],
        Warehouse= row['Warehouse'],
        Product_Category= row['Product_Category'],
        Date= row['Date'],
        Order_Demand= row['Order_Demand']
    ))
    
    if idx % 10000 == 0:
        conn.commit()

conn.commit()
conn.close()

100it [00:00, 2439.26it/s]


# Verificar

In [4]:
import pandas as pd

In [5]:
pd.set_option('display.max_rows',100)

In [15]:
pd.read_sql('SELECT * FROM prodemand LIMIT 10',conn)

,product_code,warehouse,product_category,date,order_demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500
5,Product_0979,Whse_J,Category_028,2012/4/19,500
6,Product_0979,Whse_J,Category_028,2012/6/5,500
7,Product_0979,Whse_J,Category_028,2012/6/27,500
8,Product_0979,Whse_J,Category_028,2012/7/23,500
9,Product_0979,Whse_J,Category_028,2012/8/29,500


# Nueva tabla

In [ ]:
cursor.execute(
"""
create table public.prodemand2 as select
    * from public.prodemand
    limit 10;
"""
)